In [2]:
from pprint import pprint
from pandas.core.frame import DataFrame
import pandas as pd
import json
import requests
import time
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import Process
# function to use requests.post to make an API call to the subgraph url

global df
global df_len
df = pd.read_csv('minitemp.csv').to_dict()
df_len = len(df['id'])
global count
count= 0

def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/5736048735d4ddab454e9c3adb00f5f5/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))


mint_query_template = '''
{
  mints(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

swap_query_template = '''
{
  swaps(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
'''

burn_query_template = '''
{
  burns(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_mint_subProcess(pair_address):
    try:
        query = mint_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['mints']}
    except:
        return {pair_address : ['Error Occur']}

def get_mint():
    file_path = "./mint.json"
    mint_json = {}
    try:
      p = Pool(8)
      start = time.time()
      global count
      for ret in p.imap(get_mint_subProcess,df['id'].values()):
        count = count+1
#        print("Got value",ret,"Time :",time.time()-start)
        mint_json.update(ret)
        if(count % 1000 == 0):
          print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
#          print("write file count : " + str(count))
#          with open(file_path,'w') as outfile:
#            json.dump(mint_json, outfile, indent=4)

      print('finish ' + str(count))
      with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)
      delta_t = time.time() - start
      print("Total Time :",delta_t)
      p.close()
      p.join()
      mint_json.clear()

    except Exception as e:
        print(e)


#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_swap_subProcess(pair_address):
    try:
        query = swap_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['swaps']}
    except:
        return {pair_address : ['Error Occur']}

def get_swap():
    file_path = "./swap.json"
    swap_json = {}
    try:
        p = Pool(8)
        start = time.time()
        global count
        for ret in p.imap(get_swap_subProcess,df['id'].values()):
            count = count+1
            swap_json.update(ret)
            if(count % 1000 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
 #               print("write file count : " + str(count))
 #               with open(file_path,'w') as outfile:
 #                   json.dump(swap_json, outfile, indent=4)

        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(swap_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        swap_json.clear()

    except Exception as e:
        print(e)

#############모든 pair 쌍에 대해서 Burn Query 후 결과 저장##############
def get_burn_subProcess(pair_address):
    try:
        query = burn_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['burns']}
    except:
        return {pair_address : ['Error Occur']}

def get_burn():
    file_path = "./burn.json"
    burn_json = {}
    try:
        p = Pool(8)
        start = time.time()
        global count
        for ret in p.imap(get_burn_subProcess,df['id'].values()):
            count = count+1
            burn_json.update(ret)
            if(count % 1000 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
#                print("write file count : " + str(count))
#                with open(file_path,'w') as outfile:
#                    json.dump(burn_json, outfile, indent=4)

        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(burn_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        burn_json.clear()

    except Exception as e:
        print(e)


if __name__=='__main__':
    get_mint()
    get_burn()
    get_swap()

finish 4
Total Time : 0.3850274085998535
finish 8
Total Time : 0.5965266227722168
finish 12
Total Time : 0.6931428909301758


In [3]:
# from pprint import pprint
# from pandas.core.frame import DataFrame
# import pandas as pd
# import json
# import requests
# import time
# from tqdm import tqdm
# from multiprocessing import Pool
# from multiprocessing import Process
# # function to use requests.post to make an API call to the subgraph url

# global df
# global df_len
# df = pd.read_csv('minitemp.csv').to_dict()
# df_len = len(df['id'])
# global count
# count= 0

def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/5736048735d4ddab454e9c3adb00f5f5/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))


mint_query_template = '''
{
  mints(orderBy: timestamp, orderDirection: desc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

swap_query_template = '''
{
  swaps(orderBy: timestamp, orderDirection: desc, where:{ pair: "pair_address" }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
'''

burn_query_template = '''
{
  burns(orderBy: timestamp, orderDirection: desc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_mint_subProcess(pair_address):
    try:
        query = mint_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['mints']}
    except:
        return {pair_address : ['Error Occur']}

def get_mint():
    file_path = "./mint.json"
    mint_json = {}
    try:
      p = Pool(24)
      start = time.time()
      global count
      for ret in p.imap(get_mint_subProcess,df['id'].values()):
        count = count+1
#        print("Got value",ret,"Time :",time.time()-start)
        mint_json.update(ret)
        if(count % 500 == 0):
          print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
          print("write file count : " + str(count))
          with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)

      print('finish ' + str(count))
      with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)
      delta_t = time.time() - start
      print("Total Time :",delta_t)
      p.close()
      p.join()
      mint_json.clear()

    except Exception as e:
        print(e)


#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_swap_subProcess(pair_address):
    try:
        query = swap_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['swaps']}
    except:
        return {pair_address : ['Error Occur']}

def get_swap():
    file_path = "./swap.json"
    swap_json = {}
    try:
        p = Pool(8)
        start = time.time()
        global count
        for ret in p.imap(get_swap_subProcess,df['id'].values()):
            count = count+1
            swap_json.update(ret)
            if(count % 500 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
                print("write file count : " + str(count))
                with open(file_path,'w') as outfile:
                    json.dump(swap_json, outfile, indent=4)

        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(swap_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        swap_json.clear()

    except Exception as e:
        print(e)

#############모든 pair 쌍에 대해서 Burn Query 후 결과 저장##############
def get_burn_subProcess(pair_address):
    try:
        query = burn_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['burns']}
    except:
        return {pair_address : ['Error Occur']}

def get_burn():
    file_path = "./burn.json"
    burn_json = {}
    try:
        p = Pool(16)
        start = time.time()
        global count
        for ret in p.imap(get_burn_subProcess,df['id'].values()):
            count = count+1
            burn_json.update(ret)
            if(count % 500 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
                print("write file count : " + str(count))
                with open(file_path,'w') as outfile:
                    json.dump(burn_json, outfile, indent=4)

        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(burn_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        burn_json.clear()

    except Exception as e:
        print(e)




def update_mint_json():
    try:
      p = Pool(24)
      start = time.time()
      global count
      for ret in p.imap(get_mint_subProcess,df['id'].values()):
        count = count+1
#        print("Got value",ret,"Time :",time.time()-start)
        mint_json.update(ret)
        if(count % 500 == 0):
          print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
          print("write file count : " + str(count))
          with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)

      print('finish ' + str(count))
      with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)
      delta_t = time.time() - start
      print("Total Time :",delta_t)
      p.close()
      p.join()
      mint_json.clear()

    except Exception as e:
        print(e)
if __name__=='__main__':
   get_mint()
   get_burn()
   get_swap()

finish 16
Total Time : 0.38915109634399414
finish 20
Total Time : 0.48694491386413574
finish 24
Total Time : 0.4972507953643799


In [5]:
df1 = pd.read_csv('minitemp.csv')
df1.head()

,createdAtBlockNumber,createdAtTimestamp,id,reserve0,reserve1,reserveETH,reserveUSD,totalSupply,txCount,reserve00,...,token00.id,token00.name,token00.symbol,token00.totalLiquidity,token00.txCount,token11.id,token11.name,token11.symbol,token11.totalLiquidity,token11.txCount
0,20006534,02-06-2024,0xc5494572bb74b5dc10e02cbcd5601dc8c529184b,3.982222e+08,1.186498e+00,1.186498e+00,4.477622e+03,21730.674673,16,3.982222e+08,...,0x87d831af790976dc00ff6b42b25bf46506c8ce90,PONKE,PONKE,3.982222e+08,16,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,293870.012682,168079953
1,20006517,02-06-2024,0x1408516701d1d3dec4c1461c19512b2ae7438cea,2.937388e+11,1.907053e+00,3.814106e+00,1.439432e+04,23.641066,25,2.937388e+11,...,0x0751b242b5e44ae58b963c5d1ea048e7794280ae,Rubin GPU,RUBIN,2.937388e+11,25,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,293870.012682,168079953
2,20006486,02-06-2024,0xb9a492b90d208ced3c4e53af9601499d05bdcbc3,2.028160e-11,1.000000e-18,1.000000e-18,3.700000e-15,0.000000,13,2.028160e-11,...,0x9cea2c26bcefb5e4d0e034420c4dd22655cc59fc,Pe Pe,PEPE,2.028160e-11,13,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,293870.012682,168079953
3,20006486,02-06-2024,0x41a0d92e89918581032880fa0698256cd24401e4,1.975682e+07,5.626379e+00,1.125276e+01,4.246587e+04,0.330387,181,1.975682e+07,...,0x57fe9b84dd8ab52a3944cd05b5001a0bb5799fdb,brettCoin,brettcoin,1.975682e+07,181,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,293870.012682,168079953


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   createdAtBlockNumber    4 non-null      int64  
 1   createdAtTimestamp      4 non-null      object 
 2   id                      4 non-null      object 
 3   reserve0                4 non-null      float64
 4   reserve1                4 non-null      float64
 5   reserveETH              4 non-null      float64
 6   reserveUSD              4 non-null      float64
 7   totalSupply             4 non-null      float64
 8   txCount                 4 non-null      int64  
 9   reserve00               4 non-null      float64
 10  reserve11               4 non-null      float64
 11  token0.id               4 non-null      object 
 12  token0.name             4 non-null      object 
 13  token0.symbol           4 non-null      object 
 14  token0.totalLiquidity   4 non-null      float6

# Initial Liquidity


In [12]:
with open('./mint.json', 'r') as f:
    mint_json = json.load(f)

len(mint_json)
datas = pd.read_csv('minitemp.csv',encoding='utf-8-sig').to_dict('records')


for data in datas:
    pair_id = data['id']

    if(len(mint_json[pair_id]) == 0 ):     #Mint 정보가 없는 pair는 초기 Liquidity를 0으로 한다.
        data['initial_Liquidity_ETH'] = 0
        data['initial_Liquidity_token'] = 0
        continue


    if(data['token0.symbol'] == 'WETH'):
        initial_Liquidity_ETH = mint_json[pair_id][0]['amount0']
        initial_Liquidity_token = mint_json[pair_id][0]['amount1']
    else:
        initial_Liquidity_ETH = mint_json[pair_id][0]['amount1']
        initial_Liquidity_token = mint_json[pair_id][0]['amount0']

    data['initial_Liquidity_ETH'] = initial_Liquidity_ETH
    data['initial_Liquidity_token'] = initial_Liquidity_token


for i in range(4):
    print(datas[i]['initial_Liquidity_token'])

DataFrame(datas).to_csv('minitemp1.csv',encoding='utf-8-sig')

472222221.75
558900000000
720000000
109155847.4


In [13]:
df2 = pd.read_csv('minitemp1.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4 non-null      int64  
 1   createdAtBlockNumber     4 non-null      int64  
 2   createdAtTimestamp       4 non-null      object 
 3   id                       4 non-null      object 
 4   reserve0                 4 non-null      float64
 5   reserve1                 4 non-null      float64
 6   reserveETH               4 non-null      float64
 7   reserveUSD               4 non-null      float64
 8   totalSupply              4 non-null      float64
 9   txCount                  4 non-null      int64  
 10  reserve00                4 non-null      float64
 11  reserve11                4 non-null      float64
 12  token0.id                4 non-null      object 
 13  token0.name              4 non-null      object 
 14  token0.symbol            4 non

In [14]:
dfmain = pd.read_csv('main_pair.csv')

In [16]:
dfmain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316707 entries, 0 to 316706
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   createdAtBlockNumber    316707 non-null  int64  
 1   createdAtTimestamp      316707 non-null  object 
 2   id                      316707 non-null  object 
 3   reserve0                316707 non-null  float64
 4   reserve1                316707 non-null  float64
 5   reserveETH              316707 non-null  float64
 6   reserveUSD              316707 non-null  float64
 7   totalSupply             316707 non-null  float64
 8   txCount                 316707 non-null  int64  
 9   reserve00               316707 non-null  float64
 10  reserve11               316707 non-null  float64
 11  token0.id               316707 non-null  object 
 12  token0.name             316654 non-null  object 
 13  token0.symbol           316627 non-null  object 
 14  token0.totalLiquidit

# MAIN WORK


## Graphlib

In [17]:
import requests

mint_query_template = '''
{
  mints(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s  }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

swap_query_template = '''
{
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
'''


burn_query_template = '''
{
  burns(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0

      amount1
      to
      sender
      timestamp
 }
}
'''


def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/5736048735d4ddab454e9c3adb00f5f5/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

def call_theGraph_mint(pair_id):
    mint_array = []
    timestamp = 0
    try:
      while(True):
        query = mint_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['mints']) < 1000):
          mint_array.extend(result['data']['mints'])
          break

        mint_array.extend(result['data']['mints'])
        timestamp = result['data']['mints'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return mint_array

def call_theGraph_swap(pair_id):
    swap_array = []
    timestamp = 0
    try:
      while(True):
        query = swap_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['swaps']) < 1000): # 1000개 미만이니까 끝낸다.
          swap_array.extend(result['data']['swaps'])
          break

        swap_array.extend(result['data']['swaps'])
        timestamp = result['data']['swaps'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return swap_array

def call_theGraph_burn(pair_id):
    burn_array = []
    timestamp = 0
    try:
      while(True):
        query = burn_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['burns']) < 1000): # 1000개 미만이니까 끝낸다.
          burn_array.extend(result['data']['burns'])
          break

        burn_array.extend(result['data']['burns'])
        timestamp = result['data']['burns'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_burn')
      print(e)

    return burn_array

## FeatureLIB


In [18]:
from decimal import Decimal
import decimal
from math import sqrt
from pprint import pprint as pp

def get_initial_Liquidity(token0_symbol,mint_data_transaction):
  if(token0_symbol == 'WETH'):
      initial_Liquidity_ETH = mint_data_transaction[0]['amount0']
      initial_Liquidity_token = mint_data_transaction[0]['amount1']
  else:
      initial_Liquidity_ETH = mint_data_transaction[0]['amount1']
      initial_Liquidity_token = mint_data_transaction[0]['amount0']

  return initial_Liquidity_ETH,initial_Liquidity_token

def get_initial_Liquidity_token(mint_data_transaction,index):
  if(index == 1):
    return Decimal(mint_data_transaction[0]['amount1'])
  else:
    return Decimal(mint_data_transaction[0]['amount0'])

def get_mint_mean_period(mint_data_transaction,initial_timestamp):
    count = len(mint_data_transaction)
    if(count == 0):
      return 0
    mint_time_add = 0
    for transaction in mint_data_transaction:
      mint_time_add = mint_time_add + int(transaction['timestamp']) - initial_timestamp
    return mint_time_add / count

def get_swap_mean_period(swap_data_transaction,initial_timestamp):
    count = len(swap_data_transaction)
    if(count == 0):
      return 0
    swap_time_add = 0
    for transaction in swap_data_transaction:
      swap_time_add = swap_time_add +  int(transaction['timestamp']) - initial_timestamp
    return swap_time_add / count

def get_burn_mean_period(burn_data_transaction,initial_timestamp):
    count = len(burn_data_transaction)
    if(count == 0):
      return 0
    burn_time_add = 0
    for transaction in burn_data_transaction:
      burn_time_add = burn_time_add + int(transaction['timestamp']) - initial_timestamp
    return burn_time_add / count

def swap_IO_rate(swap_data_transaction,index):
  swapIn = 0
  swapOut = 0
  if(index == 1):
    for data in swap_data_transaction:
      if(data['amount0In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn + 1
  else:
    for data in swap_data_transaction:
      if(data['amount1In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn +1

  return swapIn,swapOut

def get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction):

  swap_len = len(swap_data_transaction)
  burn_len = len(burn_data_transaction)
  #Case 1 Swap / Burn
  if(swap_len == 0 and burn_len == 0):
    return int(mint_data_transaction[-1]['timestamp'])
  #Case 2 Swap_transaction
  if(swap_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']))
  #Case 3 Burn Transaction
  if(burn_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  #Case 4
  return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))

def get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut):
  if(swap_data_transaction[j][eth_amountIn] == '0'): #amountIn
    return Decimal(swap_data_transaction[j][eth_amountOut]) * (-1)
  else:
    return Decimal(swap_data_transaction[j][eth_amountIn])

def get_swap_token(swap_data_transaction,j,index):
  if(index == 1):
    swap_amount = Decimal(swap_data_transaction[j]['amount1In'])
    swap_amount = Decimal(swap_amount) - Decimal(swap_data_transaction[j]['amount1Out'])
  else:
    swap_amount = Decimal(swap_data_transaction[j]['amount0In'])
    swap_amount = Decimal(swap_amount) - Decimal(swap_data_transaction[j]['amount0Out'])

  return swap_amount

def get_timestamp(data_transaction,index):
  try:
    return data_transaction[index]['timestamp']
  except:
    return '99999999999'

def check_rugpull(before_transaction_Eth, current_Eth):
  if ( abs(Decimal(current_Eth) / Decimal(before_transaction_Eth)) <= 0.01 ):
    if( Decimal(before_transaction_Eth) < 0 or Decimal(current_Eth) < 0):
      return False
    else:
      return True
  else:
    return False

def is_MEV(initial_Liquidity_token, swapIn_token):
  if(swapIn_token > initial_Liquidity_token * 5):  #swap rugpull
    return False    #얘는 진짜 러그풀인 경우
  else:
    return True     #MEV RugPull

def get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,index):
    if(index == 1):
      eth_amount = 'amount0'
      eth_amountIn = 'amount0In'
      eth_amountOut = 'amount0Out'
    else:
      eth_amount = 'amount1'
      eth_amountIn = 'amount1In'
      eth_amountOut = 'amount1Out'


    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)



    current_Liquidity_Eth = Decimal(mint_data_transaction[0][eth_amount])
    initial_Liquidity_token = get_initial_Liquidity_token(mint_data_transaction,index)
    i,j,k = 1,0,0

    while True:
      try:
        next_timestamp = min(get_timestamp(mint_data_transaction,i),get_timestamp(burn_data_transaction,k))

        while(get_timestamp(swap_data_transaction,j) <= next_timestamp ):
          if(get_timestamp(swap_data_transaction,j) == '99999999999'):
            break

          #
          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth + get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut)
          #print("swap {before : %s swap_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))

          if( check_rugpull(before_transaction_Eth,current_Liquidity_Eth) ):
              if( is_MEV(initial_Liquidity_token,get_swap_token(swap_data_transaction,j,index)) == False ):
                print("swap rugpull : initial token = %s / before Eth = %s / after Eth = %s swapIn_token_amount = %s"%(initial_Liquidity_token,str(before_transaction_Eth),str(current_Liquidity_Eth),get_swap_token(swap_data_transaction,j,index)))
                return get_timestamp(swap_data_transaction,j), Decimal(current_Liquidity_Eth / before_transaction_Eth) -1, True, before_transaction_Eth,current_Liquidity_Eth,'swap',[i,j-1,k]
          j = j+1

        #mint 인 경우 curruent_Eth 더하는 로직
        if(next_timestamp == get_timestamp(mint_data_transaction,i)): #mint가 최소라면, + 한다.
          if(next_timestamp == '99999999999'):  #이건 rugpull이 없는 경우
            try:
                #여기까지 온거면 rugpull이 없는 경우인데 이때 네가지 케이스에 대한 예외케이스를 정의 해야한다.
                #Case 1 Swap/Burn이 없는 경우
                if(swap_count == 0 and burn_count == 0):
                    return mint_data_transaction[-1]['timestamp'],0, False, 0,0,''
                #Case 2 Swap이 없는 경우
                if(swap_count == 0):
                    return max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']),0,False, 0,0,''
                #Case 3 Burn이 없는 경우
                if(burn_count == 0):
                    return max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']),0,False, 0,0,''
                #Case 4 Mint/Swap/Burn이 다 있지만, Rugpull이 아닌 경우
                return max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']),0,False, 0,0,''
            except:
              return 'Error occur',100.0,False,1,1
          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth + Decimal(mint_data_transaction[i][eth_amount])
          #print("mint {before : %s burn_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))
          i = i+1

        #burn 인 경우 current_Eth 빼는 로직 / 러그풀 타임스탬프 체
        else:
          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth - Decimal(burn_data_transaction[k][eth_amount])
          #print("burn {before : %s burn_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))
          if(check_rugpull(before_transaction_Eth,current_Liquidity_Eth)):
            return get_timestamp(burn_data_transaction,k), Decimal(current_Liquidity_Eth / before_transaction_Eth) -1, True, before_transaction_Eth,current_Liquidity_Eth,'burn'
          k = k+1
      except Exception as e:
        print(e)
        print('Critical Error Occur')
        return '1',0,False,1,1,'Error'

def token_index(data):
    if(data['token0.name'] == 'Wrapped Ether'):
        return 1
    else:
        return 0

def calc_LPToken_Holders(mint_data_transaction,burn_data_transaction):
  #Mint/Burn 트랜잭션을 분석해서, 해당 시점까지의 Holder들의 토큰보유량을 Dictionary로 만들어 놓는다.
  zap_list = ['0x343e3a490c9251dc0eaa81da146ba6abe6c78b2d','0x379b4609bdf93b3584cf7b64bc78199cf185f1cd', ]
  LP_Holders = {}
  for mint in mint_data_transaction:
    try:
      Holder_address = mint['to'] #to가 Mint를 한 Address
      LP_amount = Decimal(mint['liquidity'])
      LP_Holders[Holder_address] = LP_Holders[Holder_address] + LP_amount
    except:
      LP_Holders[Holder_address] = LP_amount

  for burn in burn_data_transaction:
    try:
      Holder_address = burn['sender'] #sender가 Burn을 한 Address
      if(Holder_address in zap_list):
        continue
      LP_amount = Decimal(burn['liquidity'])
      LP_Holders[Holder_address] = LP_Holders[Holder_address] - LP_amount
      if( LP_Holders[Holder_address] < 1E-17 ):
        del LP_Holders[Holder_address]


    except Exception as e:
      print(e)
  #      print("Burn execution from non-existent address [translation by MJ]")
  #      print("Holder address = %s" % Holder_address)
  return LP_Holders

def get_LP_stdev(LP_Holders):
  total_LP_amount = 0
  LP_ratio_dict = {}

  #전체 LP amount 구하기
  for LPtoken_amount in LP_Holders.values():
    total_LP_amount = total_LP_amount + LPtoken_amount

  #홀더 별 보유 비율 구하기
  for address,LPtoken_amount in LP_Holders.items():
    LP_ratio_dict[address] = (LPtoken_amount / total_LP_amount) * 100

  #보유 비율에 대해서 평균계산
  LP_avg = 100 / len(LP_Holders)

  #보유 비율에 대한 분산, 표준편차 계산
  LP_var =0
  for ratio in LP_ratio_dict.values():
    LP_var = LP_var + (ratio - Decimal(LP_avg))**2
  LP_var = LP_var / len(LP_Holders)

  LP_stdev = sqrt(LP_var)

  return LP_stdev, LP_avg, total_LP_amount

In [19]:
import os
import pandas as pd
import glob
from pandas.core.accessor import delegate_names
from pandas.core.frame import DataFrame
import requests

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


def split_csv(total_csv):
    rows = pd.read_csv(total_csv,chunksize=5000)
    file_count = 0
    for i, chuck in enumerate(rows):
        chuck.to_csv('./result/out{}.csv'.format(i),encoding='utf-8-sig',index=False)
        file_count = file_count+1
    return file_count


def merge_csv():
  input_file = r'./result/'
  output_file = r'./result/result'

  allFile_list = glob.glob(os.path.join(input_file, 'fout*')) # glob함수로 sales_로 시작하는 파일들을 모은다
  allFile_list.sort()
  for file in allFile_list:
      output_file = output_file + file[-5]
  output_file = output_file +'.csv'

  all_Data = []
  for file in allFile_list:
    records = pd.read_csv(file).to_dict('records')
    all_Data.extend(records)

  DataFrame(all_Data).to_csv(output_file,encoding='utf-8-sig',index=False)
  print("결과물 파일 : " + output_file)

In [21]:
from pandas.core.frame import DataFrame
import pandas as pd
import time
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import Process
import datetime

def switch_file(file_name):
    global datas
    datas = pd.read_csv(file_name).to_dict('records')


'''
pair_address = '0x49179a590b086ee09dacc5750cfdb312c0c73d10'
mint_data_transaction[0]
initial_timestamp
last_timestamp
active_period / (60*60*24*30)


'''

def get_feature(data):
    try:
        pair_address = data['id']

        #TheGraph API를 이용해서 하나의 페어에 대한 쌍들을 전부 메모리에 올려놓고. 시작
        mint_data_transaction = call_theGraph_mint(pair_address)
        swap_data_transaction = call_theGraph_swap(pair_address)
        burn_data_transaction = call_theGraph_burn(pair_address)

        rugpull_timestamp, rugpull_change, is_rugpull, before_rugpull_Eth, after_rugpull_Eth,rugpull_method = get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,token_index(data))


        #initial_Liquidity 의 이더와 토큰 구하기
        initial_Liquidity_Eth , initial_Liquidity_Token = get_initial_Liquidity(data['token0.symbol'],mint_data_transaction)

        # 각각의 count 구하기
        mint_count = len(mint_data_transaction)
        swap_count = len(swap_data_transaction)
        burn_count = len(burn_data_transaction)

        # Mint/Burn/Swap의 Active Period 상의 분포
        initial_timestamp = int(mint_data_transaction[0]['timestamp'])
        last_timestamp = get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction)
        active_period = last_timestamp - initial_timestamp
        mint_mean_period = int(get_mint_mean_period(mint_data_transaction,initial_timestamp))
        swap_mean_period = int(get_swap_mean_period(swap_data_transaction,initial_timestamp))
        burn_mean_period = int(get_burn_mean_period(burn_data_transaction,initial_timestamp))

        #SwapIn/SwapOut 비율
        swapIn,swapOut = swap_IO_rate(swap_data_transaction,token_index(data))


        #rugpull 이 시작부터 끝날때까지 경과한 시간
        rugpull_proceeding_time = int(rugpull_timestamp) - int(initial_timestamp)
        if(is_rugpull == False):
            rugpull_proceeding_time = 0
            rugpull_method = ''
            rugpull_timestamp = '0'
            rugpull_change = ''


        LP_Creator = mint_data_transaction[0]['to']
        #mint/burn을 분석해서 해당 시점에 LP홀더들의 보유량을 dictionary로 만든다.
        LP_Holders = calc_LPToken_Holders(mint_data_transaction,burn_data_transaction)
        LP_stdev, LP_avg, total_LP_amount = get_LP_stdev(LP_Holders)
        try:
            LP_Creator_amount = LP_Holders[LP_Creator] #해당시점에 LP초기 제공자가 가지고 있는 양
        except:
            LP_Creator_amount = 0


        #데이터 저장
        data['initial_Liquidity_Eth'] = initial_Liquidity_Eth
        data['initial_Liquidity_Token'] = initial_Liquidity_Token
        data['last_transaction_timestamp'] = last_timestamp
        data['last_transaction_date'] = datetime.datetime.fromtimestamp(int(last_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
        data['mint_count'] = mint_count
        data['swap_count'] = swap_count
        data['burn_count'] = burn_count
        data['mint_mean_period'] = mint_mean_period
        data['swap_mean_period'] = swap_mean_period
        data['burn_mean_period'] = burn_mean_period
        data['swapIn'] = swapIn
        data['swapOut'] = swapOut
        data['active_period'] = active_period
        data['rugpull_method'] = rugpull_method
        data['rugpull_timestamp'] = rugpull_timestamp
        data['rugpull_timestamp_date'] = datetime.datetime.fromtimestamp(int(rugpull_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
        data['before_rugpull_Eth'] = before_rugpull_Eth
        data['after_rugpull_Eth'] = after_rugpull_Eth
        data['rugpull_change'] = rugpull_change
        data['rugpull_proceeding_hour'] = str(rugpull_proceeding_time / 3600) + 'h'
        data['LP_Creator_amount'] = LP_Creator_amount
        data['LP_Creator_address'] = LP_Creator
        data['LP_avg'] = LP_avg
        data['LP_stdev'] = LP_stdev
        data['total_LP_amount'] = total_LP_amount
        data['is_rugpull'] = is_rugpull

        return data
    except Exception as e:
        print(e)
        return -1



if __name__=='__main__':
    createFolder('./result')
    file_name = './minitemp.csv'
    file_count = split_csv(file_name)
    out_list = []
    out_list = list(input('입력(공백단위) : ').split())

    for i in out_list:         #하나의 파일 단위로 Creator Address 불러오고, 해당 초기 유동성풀 이더값 구해온다.
        file_name = './result/out{}.csv'.format(i)
        switch_file(file_name)
        datas_len = len(datas)
        try:
            p = Pool(4)
            count = 0
            result = []
            for ret in p.imap(get_feature,datas):
                if(ret == -1):
                    count = count+1
                    continue
                count = count+1
                result.append(ret)
                if(count % 200 == 0):
                    print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
            p.close()
            p.join()
        except Exception as e:
            print(e)
        print('===================================   finish    =========================================')
        time.sleep(5)

        df = pd.DataFrame(result)
        file_name = './result/fout{}.csv'.format(i)
        df.to_csv(file_name,encoding='utf-8-sig',index=False)
        print(file_name + ' complete')
    merge_csv()

입력(공백단위) : 0
local variable 'LP_amount' referenced before assignment
local variable 'LP_amount' referenced before assignment
local variable 'LP_amount' referenced before assignment
local variable 'LP_amount' referenced before assignment
===================================   finish    =========================================
./result/fout0.csv complete


EmptyDataError: No columns to parse from file

In [22]:
tempdf = pd.read_csv("/content/result/out0.csv")

In [23]:
tempdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   createdAtBlockNumber    4 non-null      int64  
 1   createdAtTimestamp      4 non-null      object 
 2   id                      4 non-null      object 
 3   reserve0                4 non-null      float64
 4   reserve1                4 non-null      float64
 5   reserveETH              4 non-null      float64
 6   reserveUSD              4 non-null      float64
 7   totalSupply             4 non-null      float64
 8   txCount                 4 non-null      int64  
 9   reserve00               4 non-null      float64
 10  reserve11               4 non-null      float64
 11  token0.id               4 non-null      object 
 12  token0.name             4 non-null      object 
 13  token0.symbol           4 non-null      object 
 14  token0.totalLiquidity   4 non-null      float6

In [24]:
tempdf1 = pd.read_csv("/content/result/fout0.csv")
tempdf1.info()

EmptyDataError: No columns to parse from file